In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
import GPUtil
GPUtil.showUtilization()

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, sampler

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import os

In [ ]:
print(os.getcwd())
os.chdir('/opt/ml/input/data')
os.getcwd()

In [ ]:
!find . -regex ".*\.\_[a-zA-Z0-9._]+" -delete

In [ ]:
random_seed = 12
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [ ]:
train_dir_path = '/opt/ml/input/data/train/'
train_image_path = '/opt/ml/input/data/train/images/'

dt_train = pd.read_csv(train_dir_path+'train.csv')
dt_train

In [ ]:
whole_image_path = []
whole_target_label = []

for path in dt_train['path']:
    for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
        whole_image_path.append(train_image_path+path+'/'+file_name)
        whole_target_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))

In [ ]:
len(whole_image_path)

In [ ]:
def onehot_enc(x):
    def gender(i):
        if i == 'male':
            return 0
        elif i == 'female':
            return 3
    def age(j):
        j = int(j)
        if j < 30:
            return 0
        elif j >= 30 and j < 60:
            return 1
        elif j >= 60:
            return 2
    def mask(k):
        if k == 'normal':
            return 12
        elif 'incorrect' in k:
            return 6
        else:
            return 0
    return gender(x[0]) + age(x[1]) + mask(x[2])

In [ ]:
sr_data = pd.Series(whole_image_path)
sr_label = pd.Series(whole_target_label)

In [ ]:
class Dataset_Mask(Dataset):
    def __init__(self, encoding=True, midcrop=True, transform=None):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = sr_data
        self.label = sr_label
        self.transform = transform
        
        if encoding:
            self.label = self.label.apply(onehot_enc)
        
    def __len__(self):
        return len(sr_data)
    
    def __getitem__(self, idx):
        X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        y = self.label[idx]
        
        if self.midcrop:
            X = X[80:380, 42:342]
        
        if self.transform:
            return self.transform(X), y
        return X, y

In [ ]:
dataset_mask = Dataset_Mask(transform = transforms.Compose([
                                transforms.ToTensor()
                            ]))

In [ ]:
train_size = int(len(dataset_mask) * 0.8)
val_size = int(len(dataset_mask) * 0.2)

In [ ]:
train_idx, valid_idx = train_test_split(np.arange(len(dt_train)),
                                       test_size=0.2,
                                       shuffle=True,
                                       stratify=dt_train['age_range'])

In [ ]:
train_image = []
train_label = []

for idx in train_idx:
    path = dt_train.iloc[idx]['path']
    for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
        train_image.append(train_image_path+path+'/'+file_name)
        train_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))                                 

In [ ]:
valid_image = []
valid_label = []

for idx in valid_idx:
    path = dt_train.iloc[idx]['path']
    for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
        valid_image.append(train_image_path+path+'/'+file_name)
        valid_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))                                 

In [ ]:
train_data = pd.Series(train_image)
train_label = pd.Series(train_label)

valid_data = pd.Series(valid_image)
valid_label = pd.Series(valid_label)

In [ ]:
def make_class_weights(labels, class_num):
    labels = np.array(labels) 
    class_weights = np.zeros_like(labels) 
    
    _, counts = np.unique(labels, return_counts=True)
    # 각 class가 몇 번 등장하는지 count

    for cls in range(class_num):
        class_weights = np.where(labels == cls, 1/counts[cls], class_weights)
        # label이 class에 해당하면 count의 역수 적용
    return class_weights

class_num = 18

In [ ]:
mask_train_set = Dataset_Mask(data=train_data, label=train_label, transform = transforms.Compose([
                                transforms.ToTensor()
                            ]))

In [ ]:
mask_val_set = Dataset_Mask(data=valid_data, label=valid_label, transform = transforms.Compose([
                                transforms.ToTensor()
                            ]))

In [ ]:
mask_train_lables = torch.index_select(input=dataset_mask, dim=0, index=torch.tensor(mask_train_set.indices))

class_weights = torch.DoubleTensor(make_class_weights(mask_train_lables, class_num)) # np to double tensor

print(class_weights)
print(len(class_weights))

In [ ]:
batch_size = 256

sampler = sampler.WeightedRandomSampler(weights=class_weights, num_samples=len(class_weights))

In [ ]:
train_dataloader_mask = DataLoader(dataset = mask_train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader_mask = DataLoader(dataset = mask_val_set, batch_size=batch_size, shuffle=True, num_workers=2)

len(train_dataloader_mask)

In [ ]:
for epoch in range(3):
    print("epoch ", epoch)
    for i, (inputs, labels) in enumerate(train_dataloader_mask):
        if i == 3:
            break
        _, counts = np.unique(labels, return_counts=True)
        print(i,": ", counts)

In [ ]:
batch_size = 256

train_dataloader_mask_weighted = DataLoader(dataset=mask_train_set, batch_size=batch_size, sampler=sampler, num_workers=2)

len(train_dataloader_mask_weighted)

In [ ]:
for epoch in range(5):
    print("epoch ", epoch)
    for i, (inputs, labels) in enumerate(train_dataloader_mask_weighted):
        if i == 5:
            break
        _, counts = np.unique(labels, return_counts=True)
        print(i,": ", counts)

In [ ]:
basemodel_resnet34 = torchvision.models.resnet34(pretrained=True)
print('필요 입력 채널 개수', basemodel_resnet34.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', basemodel_resnet34.fc.weight.shape[0])
print(basemodel_resnet34)

In [ ]:
import math

basemodel_resnet34.fc = nn.Linear(in_features=512, out_features=class_num, bias=True)
nn.init.xavier_uniform_(basemodel_resnet34.fc.weight)
stdv = 1. / math.sqrt(basemodel_resnet34.fc.weight.size(1))
basemodel_resnet34.fc.bias.data.uniform_(-stdv, stdv)

print('필요 입력 채널 개수', basemodel_resnet34.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', basemodel_resnet34.fc.weight.shape[0])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

basemodel_resnet34.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 30

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(basemodel_resnet34.parameters(), lr=LEARNING_RATE)

In [ ]:
np.set_printoptions(precision=3)
n_param = 0
for p_idx, (param_name, param) in enumerate(basemodel_resnet34.named_parameters()):
    if param.requires_grad:
        param_numpy = param.detach().cpu().numpy()
        n_param += len(param_numpy.reshape(-1))
        print ("[%d] name:[%s] shape:[%s]."%(p_idx,param_name,param_numpy.shape))
        print ("    val:%s"%(param_numpy.reshape(-1)[:5]))
print ("Total number of parameters:[%s]."%(format(n_param,',d')))

In [ ]:
best_val_acc = 0
best_val_loss = np.inf
patience = 5
cur_count = 0

for epoch in range(NUM_EPOCH):
    basemodel_resnet34.train()
    loss_value = 0
    matches = 0
    for train_batch in train_dataloader_mask:
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outs = basemodel_resnet34(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            torch.save(basemodel_resnet34, '/opt/ml/checkpoint/resnet34/checkpoint_ep_%d.pt'% epoch)
        
        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        
        train_loss = loss_value / batch_size
        train_acc = matches / batch_size
        
        loss_value = 0
        matches = 0
    print(f"epoch[{epoch}/{NUM_EPOCH}] training loss {train_loss:.3f}, training accuracy {train_acc:.3f}")
        
    with torch.no_grad():
        basemodel_resnet34.eval()
        val_loss_items = []
        val_acc_items = []
        for val_batch in val_dataloader_mask:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = basemodel_resnet34(inputs)
            preds = torch.argmax(outs, dim=-1)
            
            loss_item = criterion(outs, labels).item()
            acc_item = (labels==preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
        val_loss = np.sum(val_loss_items) / len(val_dataloader_mask)
        val_acc = np.sum(val_acc_items) / len(mask_val_set)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            cur_count = 0
            torch.save(basemodel_resnet34, '/opt/ml/checkpoint/resnet34/checkpoint_best.pt')
        else:
            cur_count += 1
            if cur_count >= patience:
                print("Early Stopping!")
                break
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            
        print(f"[val] acc : {val_acc:.3f}, loss : {val_loss:.3f}")
        print(f"best acc : {best_val_acc:.3f}, best loss : {best_val_loss:.3f}")

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir_path = '/opt/ml/input/data/eval/'
test_image_path = '/opt/ml/input/data/eval/images/'

basemodel_resnet34 = torch.load('/opt/ml/checkpoint/resnet34/checkpoint_best.pt')
submission = pd.read_csv(test_dir_path+'info.csv')
submission.head()

In [ ]:
image_paths = [os.path.join(test_image_path, img_id) for img_id in submission.ImageID]
test_image = pd.Series(image_paths)

In [ ]:
class Test_Dataset(Dataset):
    def __init__(self, midcrop=True, transform=None):
        self.midcrop = midcrop
        self.data = test_image
        self.transform = transform
        
    def __len__(self):
        return len(test_image)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        
        if self.midcrop:
            img = img[64:448]
            
        if self.transform:
            img = self.transform(img)
            
        return img

In [ ]:
dataset = Test_Dataset(transform = transforms.Compose([
                            transforms.ToTensor()
                        ]))

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = basemodel_resnet34.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir_path, 'submission.csv'), index=False)
print('test inference is done!')